### Prepare dataset

In [5]:
# Install sentence-transformers if not already installed
!pip install -U sentence-transformers

In [6]:
import pandas as pd
import numpy as np
import pickle
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
from safetensors.torch import save_file
import os
import torch

In [7]:
brand_category = pd.read_csv('data/brand_category.csv')
categories = pd.read_csv('data/categories.csv')
offer_retailer = pd.read_csv('data/offer_retailer.csv')

#### Preprocessing brand_category table

In [8]:
brand_category.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844


1. The column receipts is not required for our use case so dropping it.
2. Converting the text data in lower case.

In [9]:
brand_category.drop(columns=['RECEIPTS'], inplace=True)
brand_category.rename(columns={'BRAND': 'brand', 'BRAND_BELONGS_TO_CATEGORY':'brand_category'}, inplace=True)
for col in brand_category.columns:
    brand_category[col] = brand_category[col].str.lower()
brand_category.head()

,brand,brand_category
0,caseys gen store,tobacco products
1,caseys gen store,mature
2,equate,hair removal
3,palmolive,bath & body
4,dawn,bath & body


#### Preprocessing categories table

In [10]:
categories.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness


1. The table category_id is a primary key to the categories table, however, it hasn't been used as foreign key in other tables so it is of little to now use to us. Hence, dropping it.
2. Convert all columns to lower case.
3. We can observe that there is a category and then there is corresponding parent category, so let's rename them accordingly and group the children category together based on parent category for simplicity.

In [11]:
categories.drop(columns=['CATEGORY_ID'], inplace=True)
categories.rename(columns={'PRODUCT_CATEGORY': 'brand_category', 'IS_CHILD_CATEGORY_TO':'parent_category'}, inplace=True)
for col in categories.columns:
    categories[col] = categories[col].str.lower()

categories = categories.groupby("parent_category", group_keys=True).apply(lambda x: x)
categories = categories[['parent_category', 'brand_category']]

categories.head()

parent_category  \
parent_category                      
alcohol         29         alcohol   
                36         alcohol   
                39         alcohol   
                40         alcohol   
                50         alcohol   

                                                    brand_category  
parent_category                                                     
alcohol         29                                            beer  
                36                                  malt beverages  
                39  hard seltzers, sodas, waters, lemonades & teas  
                40                                     hard ciders  
                50                                            wine

#### Preprocessing offer_retailer table

In [12]:
offer_retailer.head()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dil...",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",AMAZON,GATORADE


1. There are null values in the table so, the first step would be removing them, in our class we will replace them with empty strings.
2. Convert all the columns except the offer detail one to lower case.

In [13]:
offer_retailer.fillna('', inplace=True)
offer_retailer.rename(columns={'RETAILER': 'retailer', 'BRAND':'brand'}, inplace=True)
for col in [ 'retailer', 'brand']:
    offer_retailer[col] = offer_retailer[col].str.lower()

offer_retailer.head()

,OFFER,retailer,brand
0,Spend $50 on a Full-Priced new Club Membership,sams club,sams club
1,"Beyond Meat® Plant-Based products, spend $25",,beyond meat
2,Good Humor Viennetta Frozen Vanilla Cake,,good humor
3,"Butterball, select varieties, spend $10 at Dil...",dillons food store,butterball
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",amazon,gatorade


#### Merging all the dataframes we have to get the final required dataset

We need all the details associated with the offer i.e. retailer, brand and brand_category to be present in the same dataframe. So, we will just merge all those dataframes together.

In [14]:
temp_df = pd.merge(offer_retailer,brand_category, on='brand', how='left')
df = pd.merge( temp_df, categories, how='left', on = 'brand_category')
df.head()

,OFFER,retailer,brand,brand_category,parent_category
0,Spend $50 on a Full-Priced new Club Membership,sams club,sams club,NaN,NaN
1,"Beyond Meat® Plant-Based products, spend $25",,beyond meat,plant-based meat,meat & seafood
2,"Beyond Meat® Plant-Based products, spend $25",,beyond meat,frozen plant-based meat,frozen
3,"Beyond Meat® Plant-Based products, spend $25",,beyond meat,packaged meat,pantry
4,Good Humor Viennetta Frozen Vanilla Cake,,good humor,frozen desserts,frozen


As we can see, there are some null values, so we will replace them with empty strings.

In [15]:
df.fillna('', inplace=True)

#### Create corpus of offers

Let us concatenate the details of offer with correpsonding metadata so that when we search the dataset using search query ie. metadata, we will be able to easily retrieve required records.

In [16]:
corpus = []
for ind, row in df.iterrows():
    document = row['OFFER'] + ' {' + row['retailer'] + ' ' + row['brand'] + row['brand_category'] + ' ' + \
          row['parent_category'] + '}'
    corpus.append(document)
passages = corpus

#### Save the corpus for future use

Save the corpus to a pickle file. Please note that dataset here is extremely small ie. approx. 1 MB so we are storingit in a pickle file. However, for a dataset where there would have been tens of thousands of records and spans a few GB
is size, we would be using a sophisticated database system preferrably, a vector database.

In [17]:
# with open('corpus.pickle', 'wb') as f :
#    pickle.dump(corpus,f)
# with open('corpus.pickle, 'rb') as f :
#    passages = pickle.load(f)

### Create Vector Dataset

Check the length i.e. number of words in offer details so that we can use apt SentenceTransformer

In [18]:
temp_list = [len(list(x.split())) for x in df.OFFER.values]
max(temp_list)

16

We will use bi-encoders to encode all the documents and get corresponding vectors. Once we have those we can use these to find the records that are closest to search query vector.
Once we obtain those results, say top k, we will find the similarity between those top-k results and the search query using cross-encoder. And rearrange them according to the similarity scores.

So basically, we use bi-encoders to get top-k matching results from the database and cross-encoders to get the most similar ones out of those top-k.

**Why bi-encoders for retriving similar records ?** <br>
Because they are fast (faster than the cross-encoders).

**Why cross-encoders for rearraning the top-k results based on similarity ?** <br>
Because they are better at finding semantic as well as text-based (textual/syntactic) similarity than bi-encoders.

In [19]:
# Bi-Encoder to encode all passages, so that we can use it with sematic search
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256     # Truncate long passages to 256 tokens
top_k = 32                          # Number of passages we want to retrieve with the bi-encoder

# load cross encoders
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# create vector embeddings for the dataset
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

#### Save the vector database for future use

In [20]:
# tensors = {
#    "embedding": corpus_embeddings,
#}
#save_file(tensors, "embeddings.safetensors")

#tensors = {}
#with safe_open("embeddings.safetensors", framework="pt", device=0) as f:
#    for k in f.keys():
#        tensors[k] = f.get_tensor(k)

In [21]:
def search(query, top_k):

    # Search using bi-encoder #
    # Encode the search query using bi-encoder and search for relevant offers
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    query_embedding = query_embedding.cuda()
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    # Re-ranking using cross-encoder #
    # Finding and rearranging the top-k results using similarity scores obtained from cross-encoders
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-10 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:10]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-10 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    score_list, output_list = [],[]
    for hit in hits[:10]:
        score_list.append("{:.3f}".format(hit['cross-score']))
        temp_output = passages[hit['corpus_id']].replace("\n", " ")
        temp_output = list(temp_output.rsplit('{'))[0].strip()
        output_list.append(temp_output)

    dataframe = pd.DataFrame({'score': score_list, 'offers': output_list})
    dataframe.drop_duplicates(subset=['offers'], keep='first', inplace=True)
    # st.dataframe(dataframe)
    dataframe.to_csv('temp_output.csv', index=False)
    print(dataframe)

In [23]:
search('walmart', 10)


-------------------------

Top-10 Bi-Encoder Retrieval hits
	0.664	Arber, at Walmart {walmart arber }
	0.514	L’Oréal Paris Men Expert hair color, spend $9 at Walmart {walmart loreal paris hair colorhair care health & wellness}
	0.513	L'Oréal Paris Men Expert hair color, spend $19 at Walmart {walmart loreal paris hair colorhair care health & wellness}
	0.481	Gorton's at select retailers {walmart gortonsfrozen meals frozen}
	0.479	Little Bites® Spend $10 at Walmart® {walmart entenmannsbakery deli & bakery}
	0.475	Kradle, select varieties, spend $20 at Walmart {walmart kradledog supplies animals & pet supplies}
	0.472	Welch's® Fruit 'n Yogurt® Snacks, 18 count, at Walmart {walmart welchs fruit snacksfruit & vegetable snacks snacks}
	0.471	Gorton's at select retailers {walmart gortonsjerky & dried meat snacks}
	0.470	General Mills™ products, select brands, spend $7 at Convenience Stores { annies homegrown grocerycheese dairy}
	0.460	General Mills™ products, select brands, spend $35 { anni

In [24]:
search('sam club', 10)


-------------------------

Top-10 Bi-Encoder Retrieval hits
	0.743	Spend $50 on a Full-Priced new Club Membership {sams club sams club }
	0.659	Spend $110 on a Full-Priced new Plus Membership and receive an ADDITIONAL 10,000 points {sams club sams club }
	0.640	George's Farmers Market Chicken Wings, at Sam's Club {sams club georges farmers market }
	0.441	Tyson Products, select varieties, spend $20 at Sam's Club {sams club ball park frankpackaged meat pantry}
	0.425	Sign up for The Club Card or The Club+ Card full-priced membership* (New Members Only) {bjs wholesale bjs wholesalecooking & baking pantry}
	0.416	Tyson Products, select varieties, spend $20 at Sam's Club {sams club ball park frankfrozen beef frozen meat}
	0.305	Gorton's at select retailers {walmart gortonsfrozen meals frozen}
	0.300	Gorton's at select retailers {walmart gortonsjerky & dried meat snacks}
	0.292	Gorton's, at select retailers {stop & shop gortonsfrozen meals frozen}
	0.287	When you join Costco as a Gold Star

In [26]:
search('target',10)


-------------------------

Top-10 Bi-Encoder Retrieval hits
	0.593	Arber, at Target {target arber }
	0.509	L'Oreal Paris True Match Foundation at Target {target loreal paris cosmeticsmakeup beauty}
	0.509	L'Oréal Paris True Match Foundation at Target {target loreal paris cosmeticsmakeup beauty}
	0.490	L'Oréal Paris Makeup, spend $35 at Target {target loreal paris cosmeticsmakeup beauty}
	0.487	L'Oréal Paris Makeup, spend $30 at Target {target loreal paris cosmeticsmakeup beauty}
	0.447	L'Oréal Paris Excellence Hair Color at Target {target loreal paris hair colorhair care health & wellness}
	0.442	Beyond Steak™ Plant-Based seared tips, 10 ounce at Target {target beyond meatplant-based meat meat & seafood}
	0.439	L'Oréal Paris Hair color, select varieties, spend $9 at Target {target loreal paris hair colorhair care health & wellness}
	0.433	L'Oréal Paris Hair color, select varieties, spend $25 at Target {target loreal paris hair colorhair care health & wellness}
	0.430	L'Oréal Paris Hai

In [28]:
search('diapers', 10)


-------------------------

Top-10 Bi-Encoder Retrieval hits
	0.362	Butterball, select varieties, spend $10 at King Soopers {king soopers butterballnut butters & jam pantry}
	0.361	BallPark® buns, buy 2 { ball park pop upsbakery deli & bakery}
	0.347	Glad® Trash Bags, 4 OR 8 Gallon { gladcrackers snacks}
	0.341	Glad® Trash Bags, 4 OR 8 Gallon { gladprepared meals deli & bakery}
	0.337	Dr Pepper®, Regular OR Zero Sugar Strawberries and Cream 12 pack, at select retailers {united supermarkets dr peppercooking & baking pantry}
	0.322	Hellmann's® OR Best Foods® AND Sargento® Cheese Slices { hellmanns best foodsdressings pantry}
	0.315	Brita® Pitcher OR Dispenser { britahousehold supplies household supplies}
	0.314	Brita® Pitcher OR Dispenser { britawater beverages}
	0.311	Glad® Trash Bags, 4 OR 8 Gallon { gladfood storage household supplies}
	0.308	Sara Lee® bread, select varieties, buy 2 at Walmart® {walmart sara leefrozen desserts frozen}

-------------------------

Top-10 Cross-Encoder R

#### References :

1. https://www.sbert.net/docs/pretrained_models.html
2. https://www.sbert.net/docs/pretrained-models/ce-msmarco.html
3. https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2
4. https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1